# Stock Prediction using RNN and CNN

In [21]:
import pandas as pd

# Load the dataset
apple_stock_data = pd.read_csv("AAPL.csv", names=['date', 'open', 'low', 'close', 'adj close', 'volume'])
apple_stock_data.head()


,date,open,low,close,adj close,volume
Date,Open,High,Low,Close,Adj Close,Volume
2003-11-03,0.407679,0.416071,0.406786,0.413393,0.350425,302842400
2003-11-04,0.411964,0.412500,0.403393,0.409107,0.346792,249233600
2003-11-05,0.407500,0.413036,0.401250,0.411250,0.348608,322470400
2003-11-06,0.409107,0.413393,0.404464,0.412857,0.349971,397073600


In [23]:
# reprocess_data 
from sklearn.preprocessing import MinMaxScaler
import numpy as np

def preprocess_data(data):

    # Handling missing values
    data.fillna(method='ffill', inplace=True)  # Fill missing values with the previous value

    # Feature Engineering: Adding moving average as an example
    data['moving_average'] = data['close'].rolling(window=5).mean()

    # Scaling
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(data[['open', 'low', 'close', 'adj close', 'volume', 'moving_average']])

    # Sequence Creation
    def create_sequences(data, sequence_length):
        xs, ys = [], []
        for i in range(len(data) - sequence_length):
            x = data[i:(i + sequence_length)]
            y = data[i + sequence_length][2]  # Assuming 'close' price is the target
            xs.append(x)
            ys.append(y)
        return np.array(xs), np.array(ys)

    sequence_length = 60  # Number of days for the sequence
    return create_sequences(scaled_data, sequence_length)

In [24]:
# data split
def split_data(features, labels, train_size_percentage=0.8):
    # Calculate the index to split the data
    train_size = int(len(features) * train_size_percentage)

    # Split the data
    train_features = features[:train_size]
    train_labels = labels[:train_size]
    validation_features = features[train_size:]
    validation_labels = labels[train_size:]

    return (train_features, train_labels), (validation_features, validation_labels)



In [25]:
# rnn model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout, BatchNormalization, Bidirectional

def create_enhanced_rnn_model(input_shape):
    rnn_model = Sequential()

    # Adding Bidirectional LSTM layers for better temporal learning
    rnn_model.add(Bidirectional(LSTM(128, return_sequences=True), input_shape=input_shape))
    rnn_model.add(Dropout(0.25))
    rnn_model.add(BatchNormalization())

    rnn_model.add(Bidirectional(LSTM(128, return_sequences=True)))
    rnn_model.add(Dropout(0.25))
    rnn_model.add(BatchNormalization())

    rnn_model.add(Bidirectional(LSTM(128)))
    rnn_model.add(Dropout(0.25))
    rnn_model.add(BatchNormalization())

    # Adding more Dense layers for complexity
    rnn_model.add(Dense(64, activation='relu'))
    rnn_model.add(Dropout(0.25))

    rnn_model.add(Dense(32, activation='relu'))
    rnn_model.add(Dropout(0.25))

    # Output layer
    rnn_model.add(Dense(1, activation='linear'))  # Assuming a regression problem for stock price prediction

    return rnn_model



In [26]:
# cnn model
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Activation, Flatten

def create_enhanced_cnn_model(input_shape):
    cnn_model = Sequential()

    # Adjusting the number of filters and kernel size
    cnn_model.add(Conv1D(64, 5, activation='relu', input_shape=input_shape))
    cnn_model.add(MaxPooling1D(pool_size=2))
    cnn_model.add(Dropout(0.25))

    cnn_model.add(Conv1D(64, 5, activation='relu'))
    cnn_model.add(MaxPooling1D(pool_size=2))
    cnn_model.add(Dropout(0.25))

    cnn_model.add(Conv1D(64, 5, activation='relu'))
    cnn_model.add(MaxPooling1D(pool_size=2))
    cnn_model.add(Dropout(0.25))

    cnn_model.add(Flatten())

    # Adding more Dense layers
    cnn_model.add(Dense(64, activation='relu'))
    cnn_model.add(Dropout(0.25))

    cnn_model.add(Dense(32, activation='relu'))
    cnn_model.add(Dropout(0.25))

    # Output layer
    cnn_model.add(Dense(1, activation='linear'))  # Assuming a regression problem for stock price prediction

    cnn_model.compile(loss='mean_squared_error', optimizer='adam')

    return cnn_model

In [ ]:
# Training the Enhanced RNN Model
rnn_model = create_enhanced_rnn_model(train_features.shape[1:])
rnn_model.compile(loss='mean_squared_error', optimizer='adam')

rnn_history = rnn_model.fit(
    train_features, train_labels,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(validation_features, validation_labels)
)

# Training the Enhanced CNN Model
cnn_model = create_enhanced_cnn_model(train_features.shape[1:])
cnn_model.compile(loss='mean_squared_error', optimizer='adam')

cnn_history = cnn_model.fit(
    train_features, train_labels,
    batch_size=BATCH_SIZE,
    epochs=EPOCHS,
    validation_data=(validation_features, validation_labels)
)